In [ ]:
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm_notebook as tqdm
import time
import re

**Neural news headlines**

For all questions: 
@h4zzkR (telegram)

With data from russian media (Meduza) and pseudo-russian media (Panorama) and with help of ruGPT3 model we will get crazy headlines generator

# Let's get some data

This is real data from real media

In [ ]:
class MeduzaParser:
    def __init__(self):
        self.stream = 'https://meduza.io/api/v3/search?chrono=news&locale=ru&page={page}&per_page=24'
        self.headers = {'User-Agent' : "Mozilla/5.0 (X11; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0" }
        self.scrapped_titles = []
        self.processed = []

    def get_page_data(self, page):
        response = requests.get(self.stream.format(page = page), headers=self.headers).json()
        for url, data in response['documents'].items():
            try:
                self.scrapped_titles.append(data['title'])
            except KeyError:
                continue

    def parse(self, parse_range : tuple = (0, 2)):
        for i in tqdm(range(parse_range[0], parse_range[1])):
            self.get_page_data(i)
        print("[LOG] got all data")

    def preprocess(self):
        for title in tqdm(self.scrapped_titles):
            title = title.replace('\xa0', ' ')
            title = re.sub(r'[^-\а-яА-Яa-zA-Z0-9,.: ]', '', title)
            self.processed.append(title)

    def to_csv(self):
        df = pd.DataFrame(self.processed)
        df.columns = ['title']
        df.to_csv("meduza_snippet.csv")
        return df

    def run(self, range_range : tuple = (0, 2)):
        self.parse(range_range)
        self.preprocess()

In [ ]:
mp = MeduzaParser()
mp.run((1000,2000))
mp.to_csv()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



[LOG] got all data


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


,title
0,Сколько преступлений выявили в силовых структу...
1,"Почему Любовь Соболь, Дмитрия Гудкова и других..."
2,В США задержали россиянина за нелегальную пере...
3,Коммерсант: в 2019 году сорвалась четверть гос...
4,"В Бразилии найден мертвым наркоторговец, пытав..."
...,...
23995,Сможете стать президентом России
23996,На Артдокфесте сорвали показ фильма о войне на...
23997,Александр Сокуров и русская Смерть
23998,Хочу организовать городской фестиваль своими с...


And this is not real data

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving panorama_data.json to panorama_data.json
User uploaded file "panorama_data.json" with length 4023793 bytes


In [ ]:
class PanoramaOfflineParser:
    def __init__(self, path = 'panorama_data.json'):
        with open(path) as json_file:
            self.json_file = json.load(json_file)
        self.processed = []

    def run(self):
        for msg in self.json_file['messages']:
            try:
                hl = msg['text'][0]
            except Exception:
                if isinstance(msg['text'], str):
                    hl = msg['text']
            if isinstance(hl, str):
                hl = hl[:hl.find('\n')]
                if (len(hl) != 0):
                    hl = re.sub(r'[^-\а-яА-Яa-zA-Z0-9,.: ]', '', hl)
                    self.processed.append(hl)
      
    def to_csv(self):
        df = pd.DataFrame(self.processed)
        df.columns = ['title']
        df.to_csv("panorama_snippet.csv")
        return df

In [ ]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

from google.colab import drive
drive.mount('/gdrive')
%ls /gdrive

meduza_path = '/gdrive/MyDrive/neurama/meduza_snippet(1).csv'
panorama_path = '/gdrive/MyDrive/neurama/panorama_data.json'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
MyDrive/  Shareddrives/


In [ ]:
pp = PanoramaOfflineParser(panorama_path)
pp.run()
pp.to_csv()

,title
0,Конгресс США по ошибке проголосовал за включен...
1,Инициативные россияне создали движение А мы и ...
2,UBER создаст в Румынии первую в мире сеть гуже...
3,Престарелый чернокожий экс-работник General Mo...
4,Минобороны построило в Анапе детскую площадку ...
...,...
6856,Навальный станет лидером России в Civilization 6
6857,Накануне Дня Победы из алтаря храма Вооружнных...
6858,В Белоруссии объявили о начале всеобщей коллек...
6859,Российские торрент-трекеры обязали раздавать п...


In [ ]:
ms = pd.read_csv(meduza_path)
ps = pd.read_csv('panorama_snippet.csv')

ms.reset_index(drop=True, inplace=True)
ps.reset_index(drop=True, inplace=True)

df = pd.concat([ms, ps])
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

from sklearn.utils import shuffle
df = shuffle(df).reset_index(drop=True)

df.to_csv('snippet.csv')

In [ ]:
df

,title
0,"Саудовские генетики вырастили верблюда, приспо..."
1,Телеканал Дождь в новогоднюю ночь показал выст...
2,Путин подписал законы о повышении НДС и о новы...
3,Мальчик русский: фильм ученика Сокурова о Перв...
4,Полковник Владимир Квачков вышел на свободу
...,...
30856,"Правда, что таксисты работают сутками и зараба..."
30857,Создателя 10-часового видео с белым шумом обви...
30858,Участники акции против строительства храма в ц...
30859,Есть много белых пятен. Как госорганы объясняю...


# Char-based generation with RNNs

In [ ]:
df = pd.read_csv('snippet.csv')

In [ ]:
maxlen = df['title'].str.len().max()
print(f'Title max length is {maxlen}')

Title max length is 249.0


In [ ]:
drop_factor = 70

old_size = df.shape[0]
df = df[df['title'].str.len() < drop_factor]
df['title'] = df['title'].str.lower()
new_size = df.shape[0]

print(f"Size before cutting: {old_size}, size after cut: {new_size}")

Size before cutting: 3318, size after cut: 3318


In [ ]:
text = df["title"].str.cat(sep='\n')
text_size = len(text)
print('Text Size: %d' % text_size)

Text Size: 188593


In [ ]:
from pickle import dump

chars = list(set(text))
char2int = {c : i for i, c in enumerate(chars)}
int2char = {i : c for i, c in enumerate(chars)}


dump(char2int, open('mapping.pkl', 'wb'))

vocab_size = len(char2int)
print(f"vocab size: {vocab_size}")

encoded_text = [char2int[char] for char in text]
encode_size = len(encoded_text)

print(f"encode size: {encode_size}")

eos = char2int['\n']
int2char[eos]

vocab size: 74
encode size: 188593


'\n'

In [ ]:
seqlen = 10
batchsize = 512
batchnum = int((encode_size - seqlen) / batchsize)

from keras.utils import to_categorical

def myGenerator():
    while 1:
        for i in range(batchnum): 
            X_batch = []
            y_batch = []
            for j in range(batchsize):
                X_batch.append(encoded_text[i*batchsize+j:i*batchsize+j+seqlen])
                y_batch.append(encoded_text[i*batchsize+j+seqlen:i*batchsize+j+seqlen+1])
                
            X_batch = np.array([to_categorical(x, num_classes=vocab_size) for x in X_batch])
            y_batch = np.array(to_categorical(y_batch, num_classes=vocab_size))

            yield (X_batch, y_batch)

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, SimpleRNN
from keras.models import Model

model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(seqlen, vocab_size)))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(vocab_size))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 512)           1202176   
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 256)           787456    
_________________________________________________________________
lstm_5 (LSTM)                (None, 74)                97976     
_________________________________________________________________
dense_1 (Dense)              (None, 74)                5550      
Total params: 2,093,158
Trainable params: 2,093,158
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# my_generator = myGenerator()
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit_generator(my_generator, steps_per_epoch = batchnum, epochs = 50, verbose=1)

Epoch 1/50
  3/368 [..............................] - ETA: 13s - loss: 0.6982 - accuracy: 0.7904

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


368/368 [==============================] - 10s 26ms/step - loss: 0.6626 - accuracy: 0.8080
Epoch 2/50
368/368 [==============================] - 9s 26ms/step - loss: 0.6452 - accuracy: 0.8119
Epoch 3/50
368/368 [==============================] - 9s 26ms/step - loss: 0.6282 - accuracy: 0.8178
Epoch 4/50
368/368 [==============================] - 9s 26ms/step - loss: 0.6027 - accuracy: 0.8261
Epoch 5/50
368/368 [==============================] - 10s 26ms/step - loss: 0.5728 - accuracy: 0.8367
Epoch 6/50
368/368 [==============================] - 10s 26ms/step - loss: 0.5448 - accuracy: 0.8466
Epoch 7/50
368/368 [==============================] - 10s 26ms/step - loss: 0.5179 - accuracy: 0.8556
Epoch 8/50
368/368 [==============================] - 10s 26ms/step - loss: 0.4974 - accuracy: 0.8621
Epoch 9/50
368/368 [==============================] - 10s 26ms/step - loss: 0.4803 - accuracy: 0.8675
Epoch 10/50
368/368 [==============================] - 10s 26ms/step - loss: 0.4718 - accuracy: 

In [ ]:
from pickle import load
from keras.models import load_model
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
import random
 
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    for _ in range(n_chars):
        encoded = [mapping[char2] for char2 in in_text]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        encoded = to_categorical(encoded, num_classes=len(mapping))
        probs = model.predict_proba(encoded)
        yhat = random.choices(range(0,vocab_size), weights=probs[0], k=1)[0]
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        in_text += out_char
        if char =="\n":
            break
    return in_text

print(generate_seq(model, char2int, seqlen, 'навальный ', 50))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


навальный выдал кадры из чтобзк-в сорсал милай 



# ruGPT2(3) transformer


Теперь играем по-крупному

In [ ]:
!pip install torch==1.4.0
!pip3 install transformers==3.5.0

In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

fatal: destination path 'ru-gpts' already exists and is not an empty directory.


In [ ]:
!mkdir models/

## Data preparings

In [ ]:
df['title'] = df['title'].apply(lambda row: '<s>' + str(row) + '</s>')
text = ''
for (_, row) in df.iterrows():
  text += row.title

In [ ]:
text[0:1000]

In [ ]:
text_file = open("train.txt", "w")
text_file.write(text)
text_file.close()

## Model training

In [ ]:
    # --do_eval \
    # --eval_data_file=valid.txt \

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/ \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 2 \
    --block_size 2048 \
    --overwrite_output_dir

2021-05-14 09:47:05.889663: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/14/2021 09:47:08 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
05/14/2021 09:47:08 - INFO - filelock -   Lock 140146140535504 acquired on /root/.cache/torch/transformers/06f48b6b3173390d047e15d691fda67ae4ea7733a5eea4b6e0115f5099c4e700.b5cdfa39c63384f94159c36bc9042660c747cea5cf520b43d543bd2c68b3164d.lock
Downloading: 100% 608/608 [00:00<00:00, 522kB/s]
05/14/2021 09:47:08 - INFO - filelock -   Lock 140146140535504 released on /root/.cache/torch/transformers/06f48b6b3173390d047e15d691fda67ae4ea7733a5eea4b6e0115f5099c4e700.b5cdfa39c63384f94159c36bc9042660c747cea5cf520b43d543bd2c68b3164d.lock
05/14/2021 09:47:09 - INFO - filelock -   Lock 140146140500560 acquired on /root/.cache/torch/transformers/1b36eeb1fd7b3a6ec11bf46bde2c38e7e68f71ec774694b9e886c86001aab35d.c483bc344

In [ ]:
!python ru-gpts/generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=models/ \
    --k=10 \
    --p=0.98 \
    --length=100 \
    --repetition_penalty=3

2021-05-07 09:54:52.706225: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/07/2021 09:55:00 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=10, length=100, model_name_or_path='models/essays', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.98, padding_text='', prompt='', repetition_penalty=3.0, seed=42, stop_token='</s>', temperature=1.0, xlm_language='')
Context >>> православные храмы станут
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
православные храмы станут музеями
Context >>> В России
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
В России за год выявили более 500 новых случаев COVID-19
Context >>> Москва
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
Москва
Context >>> Песков объявил
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
Песков объяв

In [ ]:
!python ru-gpts/generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=models/ \
    --k=10 \
    --p=0.98 \
    --length=100 \
    --repetition_penalty=3

2021-05-07 10:11:32.442401: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/07/2021 10:11:41 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=10, length=100, model_name_or_path='models/essays', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.98, padding_text='', prompt='', repetition_penalty=3.0, seed=42, stop_token='</s>', temperature=1.0, xlm_language='')
Context >>> В России признали инагентом
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
В России признали инагентом вируса коронавируса
Context >>> В США установили
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
В США установили рекорд по смертности от коронавируса
Context >>> Россия закрывает
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
Россия закрывает границы с Польшей, Литвой и Белоруссией
Context >>> Лукашенко опроверг слухи


In [ ]:
!python ru-gpts/generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=models/ \
    --k=10 \
    --p=0.98 \
    --length=100 \
    --repetition_penalty=3

2021-05-13 06:40:03.375403: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
05/13/2021 06:40:12 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=10, length=100, model_name_or_path='models/', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.98, padding_text='', prompt='', repetition_penalty=3.0, seed=42, stop_token='</s>', temperature=1.0, xlm_language='')
Context >>> Путин поручил
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
Путин поручил проверить соблюдение требований закона о тишине в российских школах
Context >>> Меркель сообщила
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
Меркель сообщила о гибели в ДТП двух десятков россиян
Context >>> Traceback (most recent call last):
  File "ru-gpts/generate_transformers.py", line 268, in <module>
    main()
  File "ru-gpts/generate_transformers.py", line 213, in main
    

In [ ]:
!mv models /gdrive/MyDrive/neurama/models

# Production

In [ ]:
!pip install python-telegram-bot --upgrade

     |████████████████████████████████| 460kB 7.8MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 


In [ ]:
import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

from transformers import AutoTokenizer, AutoModelWithLMHead

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)

class Bot:
    def __init__(self, model_path):
        self.model_path = model_path
        self.tokenizer = GPT2Tokenizer.from_pretrained(self.model_path)
        self.model = GPT2LMHeadModel.from_pretrained(self.model_path)

        # self.tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
        # self.model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
        self.model.cuda()

        self.updater = Updater(token='')
        self.dispatcher = self.updater.dispatcher

    def bot_response(self, input):
        from random import choice
        input = self.tokenizer.encode(input, return_tensors="pt")

        max_leng = choice([50, 100, 130, 150])
        top_p = choice([0.9, 0.92, 0.95, 0.98])
        out = self.model.generate(input.cuda(), max_length=max_leng, repetition_penalty=3.0, do_sample=True, top_k=15, top_p=top_p, temperature=2.0)
        out = self.tokenizer.decode(out[0])
        return out[:out.find('</s>')]

    def textMessage(self, update: Update, _: CallbackContext):
        message = update.message.text
        response = self.bot_response(message)
        update.message.reply_text(response)

    def startCommand(self, update: Update, _: CallbackContext) -> None:
        user = update.effective_user
        update.message.reply_markdown_v2(
            fr'Hi {user.mention_markdown_v2()}\!',
            reply_markup=ForceReply(selective=True),
        )

    def start(self):
        self.dispatcher.add_handler(CommandHandler('start', self.startCommand))
        self.dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, self.textMessage))
        self.updater.start_polling()
        self.updater.idle()

In [ ]:
bot = Bot('/gdrive/MyDrive/neurama/models')

# Default

In [ ]:
bot.bot_response('Путин заявил о')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Путин заявил о том, что его компания не будет участвовать в торгах с «Росатомом»\nhttps://ria.ru/politics20181203/. По мнению Путина это приведет к ухудшению ситуации на Украине и потере контрольного пакета акций холдинга "Росэнергоатом", пишет ТАСС со ссылкой источник агентства."Если вы считаете Россию участником переговоров или тендера - пожалуйста" [из Кремля], то я вас очень прошу быть бдительным: мы будем смотреть все ваши предложения! Это же очевидно... В таком случае ничего хорошего для компании (как ее учредителя) нет..." //источник http//kprf-irina'

In [ ]:
bot.bot_response('В центре Москвы')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'В центре Москвы.  Это самый красивый храм во всей Москве! \n\n Выезд в Москву: от МКАД (5 км) - до Тимирязевского проспекта или около "Борисовой Якитории". 15 минут по Дмитровскому направлению, далее пешком 10-15 метров на запад к станции метро Спортивная улица... и еще через 20 километров выходите под Рязанским шоссе.... после чего переходим Калужское поле с левой стороны дороги..... дальше налево!!! :) Припарковав авто возле дома 1 можно сразу попасть ко дворнику чтобы забрать мусор))) На территории парковка тоже платная для жителей всех районов района!!!! ;) Ну а потом выезжаем уже домой – все чисто.. )) А теперь мы пойдем осматривать новый дом со смотрово'

In [ ]:
bot.bot_response('Илон Маск')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Илон Маск и его сын-близнец\nЛена Шелестова.  \\tСлучайно ли они с Ионой встречались, если предположить? Как оказалось - да! Но кто тогда был на их свадьбе в Москв'

# Fine-tuned

In [ ]:
bot.bot_response('Путин заявил о')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Путин заявил о планах продать часть земли в Китае'

In [ ]:
bot.bot_response('В центре Москвы')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'В центре Москвы задержано более тысячи человек. Ранее на их задержания вышли представители СК, УВД столицы  и ГИБДД города Махачкалы.'

In [ ]:
bot.bot_response('Илон Маск')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Илон Маск объявил об уходе с поста главы Роскосмоса'

# New Section

In [ ]:
bot.start()

2021-05-14 10:58:21,541 - apscheduler.scheduler - INFO - Scheduler started
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
